# Ant Colony Optimization based Itinerary Recommender

## "The title is pretty much self-explanatory. We will be following the ACO framework shown below:"

###### ACO Framework 

% Create Graph

% Draw Graph

%% ACO algorithm

    % Define Initial Parameters
    
    % Main loop
    
    for i = 1 to max_iter:
        % Create ants
        % Calculate fitness value of all ants
        % Find the best ant 
        % Update the pheromone matrix
        % Implement Evaporation
        % Display results 
    end

In [178]:
# Import the libraries 

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [179]:
# Import the dataset
top_dests_ktm=pd.read_csv("../../Datasets/ktm_dataset_updated_genre_column.csv")

In [180]:
print(top_dests_ktm.head())

   dest_id                 title                                   genre  \
0        1      Boudhanath Stupa  history:art_and_architecture:religious   
1        2  Swayambhunath Temple  history:art_and_architecture:religious   
2        3  Pashupatinath Temple  history:art_and_architecture:religious   
3        4     Chandragiri Hills                                  nature   
4        5       Kopan Monastery  history:art_and_architecture:religious   

     latitude  longitude  
0  27.7215062  85.359809  
1  27.7149298  85.288146  
2  27.7104461  85.346503  
3  27.6710496  85.262664  
4  27.7425438  85.362208  


In [181]:
# select the first 95 destinations as they're the destinations having the latitude and longitude currently
top_dests_ktm=top_dests_ktm[:95]

In [182]:
print(top_dests_ktm.tail())

    dest_id                       title  \
90       91     Ghanta Ghar Clock Tower   
91       92  Jamchen Lhakhang Monastery   
92       93                    Big Bell   
93       94       Mrigasthali Deer Park   
94       95            Universal Crafts   

                                     genre    latitude  longitude  
90     history:art_and_architecture:nature   27.707477  85.314711  
91  history:art_and_architecture:religious  27.7215058  85.359192  
92     history:art_and_architecture:nature  27.7268583  85.300625  
93                        nature:adventure   27.711512  85.349772  
94  history:art_and_architecture:religious  27.7173331  85.381252  


In [183]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests_ktm[['title','latitude','longitude']])

                         title    latitude  longitude
0             Boudhanath Stupa  27.7215062  85.359809
1         Swayambhunath Temple  27.7149298  85.288146
2         Pashupatinath Temple  27.7104461  85.346503
3            Chandragiri Hills  27.6710496  85.262664
4              Kopan Monastery  27.7425438  85.362208
..                         ...         ...        ...
90     Ghanta Ghar Clock Tower   27.707477  85.314711
91  Jamchen Lhakhang Monastery  27.7215058  85.359192
92                    Big Bell  27.7268583  85.300625
93       Mrigasthali Deer Park   27.711512  85.349772
94            Universal Crafts  27.7173331  85.381252

[95 rows x 3 columns]


In [184]:
# # let's plot the graph again 
# %matplotlib inline 
# plt.figure(figsize=(12,12)) 

# for lat,long in zip(top_dests_ktm['latitude'],top_dests_ktm['longitude']):
    
#     for another_lat,another_long in zip(top_dests_ktm['latitude'],top_dests_ktm['longitude']):
        
#         # create a list holding the two latitude
#         lat_list=[lat,another_lat]
#         # also create a list holding the two longitudes 
#         long_list=[long,another_long]
#         plt.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
# #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')

# for title,x,y in zip(top_dests_ktm['title'],top_dests_ktm['latitude'],top_dests_ktm['longitude']):

#     label = "{}".format(title)

#     plt.annotate(label, # this is the text
#                  (x,y), # these are the coordinates to position the label
#                  textcoords="offset points", # how to position the text
#                  xytext=(0,10), # distance from text to points (x,y)
#                  ha='center') # horizontal alignment can be left, right or center
# plt.show()

In [185]:
# let's just pick n=10 for now for better visualization
n=10 #no of destinations from dataset to consider 
top_n_dests=top_dests_ktm[:n]
print(top_n_dests)

# actual graph size
# as we will be considering an arbitrary location as starting and ending point

# define a starting point 
# extract the current location
import geocoder
g = geocoder.ip('me')
starting_point_name=str(g.city)+"(Home)"
starting_point=(starting_point_lat,starting_point_long)=tuple(g.latlng)
starting_point_index=n

   dest_id                    title  \
0        1         Boudhanath Stupa   
1        2     Swayambhunath Temple   
2        3     Pashupatinath Temple   
3        4        Chandragiri Hills   
4        5          Kopan Monastery   
5        6                   Thamel   
6        7         Garden of Dreams   
7        8      Namo Buddha (Stupa)   
8        9  Kathmandu Durbar Square   
9       10                     Asan   

                                               genre    latitude  longitude  
0             history:art_and_architecture:religious  27.7215062  85.359809  
1             history:art_and_architecture:religious  27.7149298  85.288146  
2             history:art_and_architecture:religious  27.7104461  85.346503  
3                                             nature  27.6710496  85.262664  
4             history:art_and_architecture:religious  27.7425438  85.362208  
5                                            history  27.7150086  85.307894  
6                       

In [186]:
# remove the duplicate entry

print(len(top_n_dests['latitude'].unique()))
print(len(top_n_dests['longitude'].unique()))

#print out the duplicated latitude,longitude
duplicate_lat_long = top_n_dests[top_n_dests.duplicated(['latitude','longitude'])]
print(duplicate_lat_long)

print(top_n_dests[top_n_dests['latitude']=='27.7154575'])

10
10
Empty DataFrame
Columns: [dest_id, title, genre, latitude, longitude]
Index: []
Empty DataFrame
Columns: [dest_id, title, genre, latitude, longitude]
Index: []


In [187]:
def plot_graph(ax,fig):
    """
        Function to plot the graph of matrix for every edge between the given nodes
    """

    for lat,long in zip(top_n_dests['latitude'],top_n_dests['longitude']):
        for another_lat,another_long in zip(top_n_dests['latitude'],top_n_dests['longitude']):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]

            ax.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            fig.canvas.draw()
            
    #adding label to each node
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):

        label = "{}".format(title)

        ax.annotate(label, # this is the text
                     (x,y), # these are the coordinates to position the label
                     textcoords="offset points", # how to position the text
                     xytext=(-5,5), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or center



In [188]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n+1,n+1))

# 
def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1
                     

        
        #fill out the last column (indicating distance with the starting point)
        distance_between_the_place_and_starting_point=np.sqrt((lat-starting_point_lat)**2+(long-starting_point_long)**2)
        graph[i][j]=distance_between_the_place_and_starting_point
        
        # also meanwhile fill out the last row
        # (indicating distance from the starting point)
        graph[n][i]=distance_between_the_place_and_starting_point

        #increment i
        i+=1
    graph[n][n]=0


In [189]:
create_graph()
print(graph)

[[0.         0.07196422 0.01730239 0.10946671 0.021174   0.05232003
  0.04806329 0.2654699  0.05799622 0.05155153 0.06400563]
 [0.07196422 0.         0.05852919 0.05074233 0.07904304 0.01974826
  0.02418316 0.32390089 0.01954512 0.02327766 0.04634658]
 [0.01730239 0.05852919 0.         0.09263384 0.03573404 0.03887774
  0.03438489 0.27060331 0.04250484 0.03642277 0.04678706]
 [0.10946671 0.05074233 0.09263384 0.         0.12255802 0.06307234
  0.06573315 0.3309644  0.05333185 0.05989373 0.05183403]
 [0.021174   0.07904304 0.03573404 0.12255802 0.         0.06089542
  0.05741827 0.27613055 0.06933144 0.06272094 0.0815682 ]
 [0.05232003 0.01974826 0.03887774 0.06307234 0.06089542 0.
  0.00450891 0.30640256 0.0113667  0.00786942 0.03892287]
 [0.04806329 0.02418316 0.03438489 0.06573315 0.05741827 0.00450891
  0.         0.30209226 0.01267828 0.00696979 0.03757339]
 [0.2654699  0.32390089 0.27060331 0.3309644  0.27613055 0.30640256
  0.30209226 0.         0.30450329 0.30086202 0.28435665]


In [190]:
n=n+1 #important as we have updated by graph by 1 row and 1 col

In [191]:
# Define the initial parameters of ACO
max_iter=100 #max no of iterations
ant_no=10 #number of ants 

#initial pheromone concentration
tau_0=10*1/(n*np.mean(graph))

#initial pheromone matrix
tau = tau_0*np.ones((n,n))
print("Initial pheromone matrix (Tau):")
print(tau)

#desirability/quality of each edge,eta
eta=1/graph #inverse of cost (distance)
print("Desirability matrix (Eta):")
print(eta)

rho=0.05 #evaporation rate
alpha=2 #pheromone exponential parameter
beta=1 #edge desirabiltu/quality exponential parameter


Initial pheromone matrix (Tau):
[[10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.90589249 10.90589249 10.90589249 10.90589249 10.90589249 10.90589249
  10.90589249 10.90589249 10.90589249 10.90589249 10.90589249]
 [10.9058

/tmp/ipykernel_102991/1501364078.py:14: RuntimeWarning: divide by zero encountered in true_divide
  eta=1/graph #inverse of cost (distance)


In [192]:
print(tau[2,:]**alpha)

[118.93849107 118.93849107 118.93849107 118.93849107 118.93849107
 118.93849107 118.93849107 118.93849107 118.93849107 118.93849107
 118.93849107]


In [193]:
print(eta[2,:]**beta)

[57.79547534 17.08549118         inf 10.79519087 27.9845177  25.72165684
 29.08254106  3.69544626 23.52673409 27.45535014 21.37342913]


## Now moving on to the main loop of ACO

In [194]:
# defining a function to create the ant colony 
def create_colony(graph,node_no,ant_no,tau,eta,alpha,beta,initial_node):
    """
        creates an ant colony
    """
    # create a list called 'colony'
    # inside it, there will be ants ( represented by the indices)
    # each position contains other list called 'tour' that contains
    # the tour completed by each ant
    colony=[[] for _ in range(ant_no)] #size of the colony=no of ants
    
    # Run iterations equal to the number of ants 
    for i in range(ant_no):
        
        # define the initial node
        # we should take it as a parameter actually 
        # but let's take it random for now
        # initial_node=np.random.randint(0,node_no)
        
        # nope we changed our mind 
        initial_node=initial_node
        colony[i].append(initial_node)
        
        print(colony)
        
        for j in range(node_no-1):
            #to choose the rest of the nodes
            
            #current node is the last node in the list
            current_node=int(colony[i][-1])
#             print("current node:",current_node)
            
            #use the following formula to calculate the probability 
            p_all_nodes=(tau[current_node,:]**alpha)*(eta[current_node,:]**beta)
#             print("Prob:",p_all_nodes)
            
            #make the probability of the visited nodes 0
#             print("colony:",colony)
            for node in colony[i]:
                p_all_nodes[node]=0
                
            #calculate the final probability
            p=p_all_nodes/np.sum(p_all_nodes)
#             print("Sum prob:",np.sum(p_all_nodes))
#             print("final prob:",p)
            
            #Note: 'p'is a vector having size (1,node_no)   
#             print("probability vector:",p)
            
            #select the next node based on roulette wheel
            next_node=roulette_wheel(p)
            
            #append next node to the tour 
            #made by the ant in the colony
            colony[i].append(next_node)
            
        # to complete the tour, 
        # add the initial node to the end as well
        
        colony[i].append(initial_node)
    return colony

In [195]:
# let's implement the roulette wheel function
def roulette_wheel(prob_vec):
    """
        Selects the next node based on 
        the cumulative sum of the probabilities 
        in the probability vector
    """
    #cumulative version of the prob_vec (p)
    
    cum_sum_p=np.cumsum(prob_vec)
    
    #choose a random value from 0 to 1
    random_val=np.random.rand()
#     print("random value:",random_val)
    
    #choose the first index as next_node in the probability vector
    #that has value more or equal to the random_val
    
#     print("cum sum:",cum_sum_p)
    
    next_node = np.argwhere(cum_sum_p>=random_val)[0][0]
    
    #return the next_node
    return next_node
    

### Now. let's define a function that calculates the fitness value of the route calculated

In [196]:
def fitness_function(tour,graph):
    """
        calculates the fitness of the tour
        > basically, it's the sum of cost of all edges
        in the tour.
    """
    fitness=0
    
    for i in range(len(tour)-1):
        
        current_node=tour[i] # current node
        next_node=tour[i+1] # next node
        
        # add the cost of current edge (current_node->next_node)
        # to the overall fitness
        
        fitness = fitness + graph[current_node][next_node]
        
    return fitness

### And a function to draw Pheromone graph

In [197]:
def draw_pheromone(ax,fig,tau,graph):
    """
        Draws pheromone graph
    """
    ax.clear()

    # Maximum value of pheromone
    max_tau=np.max(tau)
    # Minimum value of pheromone
    min_tau=np.min(tau)
    
    # Normalize the tau matrix
    tau_normalized = (tau - min_tau)/(max_tau - min_tau)
    for lat,long,i in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):
        for another_lat,another_long,j in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]
            ax.plot(lat_list,long_list,color=(0,0,1-tau_normalized[i][j]), marker='o', linestyle='solid',linewidth=10*tau_normalized[i][j]+0.5, markersize=8,mfc = 'white')
            fig.canvas.draw()
            fig.canvas.flush_events()

    
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
    
            

### Also, a function to draw the best tour 

In [198]:
def draw_best_tour(ax,fig,best_tour,graph):
    """
        function to draw the best tour
    """
    ax.clear()

    for i in range(len(best_tour)-1):
        #extract the current node and the next node
        current_node=best_tour[i]
        next_node=best_tour[i+1]
        
        #draw a line between current node and next node
        
        #latitude and longitude of the current node
        lat_current,long_current=top_n_dests.iloc[current_node]['latitude'],top_n_dests.iloc[current_node]['longitude']
        
        #latitude and longitude of the next node
        lat_next,long_next=top_n_dests.iloc[next_node]['latitude'],top_n_dests.iloc[next_node]['longitude']
        
        lat_list=[lat_current,lat_next]
        long_list=[long_current,long_next]
        
        plt.plot(lat_list,long_list,color='green', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
        fig.canvas.draw()
        fig.canvas.flush_events()

        
    #for annotation
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

### Also , a function to update the pheromone matrix

In [199]:
def update_pheromone(tau,colony,fitness_list):
    """
        function to update the pheromone matrix
    """
    node_no=len(colony[1])
    ant_no=len(colony)
    
    for i in range(ant_no):
        # for each ant
        for j in range(node_no-1):
            # for each node in the tour 
            current_node=colony[i][j]
            next_node=colony[i][j+1]
            
            tau[current_node,next_node]=tau[current_node,next_node]+1/fitness_list[i]
            tau[next_node,current_node]=tau[next_node,current_node]+1/fitness_list[i]
    
    return tau

## Main ACO loop 

In [200]:
# Main loop

best_fitness=np.inf
best_tour=[]

# %matplotlib notebook

# fig = plt.figure(figsize=(10,10))
# ax1 = fig.add_subplot(131)
# ax2 = fig.add_subplot(132)
# ax3 = fig.add_subplot(133)

# ax1.title.set_text('Initial Mesh')
# ax2.title.set_text('Pheromone Graph Plot')
# ax3.title.set_text('Best Tour Plot')
# plt.ion()

# # plt.title("Pheromone graph plot:")
# fig.show()
# fig.canvas.draw()

#let's draw the initial graph (mesh) first
# plot_graph(ax1,fig)

for i in range(max_iter):
    # create ant colony
    colony=[] # store it as a list
    colony=create_colony(graph,n,ant_no,tau,eta,alpha,beta,starting_point_index)
    print(f"Iteration #{i}:")
#     print(colony)
    
    # initializing fitness_list
    fitness_list=[0]*ant_no
    
    # calculate the fitness value of all ants
    for ant_i in range(ant_no):
        fitness_list[ant_i]=fitness_function(colony[ant_i],graph)
        
#     print(fitness_list)
    
    # find the best ant 
    min_value=np.min(fitness_list)
    min_index=np.argmin(fitness_list) #best ant
    
    if min_value<best_fitness:
        # replace best_fitness even smaller min_value is found
        best_fitness=min_value
        best_tour=colony[min_index] #tour of the best ant
        
    print("Best fitness: ",best_fitness)
    print("Best tour: ",best_tour)
    
    # update phermone matrix
    tau=update_pheromone(tau,colony,fitness_list)
    
    # print("Updated pheromone matrix:",tau)
    
    # apply evaporation
    tau=(1-rho)*tau
    
    # print("Updated pheromone matrix:",tau)
    
    # plot the pheromone graph
    # draw_pheromone(ax2,fig,tau,graph)
    
    # plot the best tour
    # draw_best_tour(ax3,fig,best_tour,graph)

#     plt.subplot(1,3,1)
#     %matplotlib tk
#     plt.figure(1)
    
#     draw_pheromone(tau,graph)

    
    
# drawing the best tour plot 
# draw_best_tour(ax3,fig,best_tour,graph)

# drawing end pheromone graph
# draw_pheromone(ax2,fig,tau,graph)

# display the names of destinations in the last tour
print("Best tour obtained:")
print(best_tour)
for i in best_tour:
    if i==starting_point_index:
        print(starting_point_name,end='')
        print("--->",end='')
    else:
        print(top_n_dests.iloc[i]["title"],end='')
        print("--->",end='')

[[10], [], [], [], [], [], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10], [], [], [], [], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10, 8, 9, 1, 6, 3, 0, 2, 4, 5, 7, 10], [10], [], [], [], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10, 8, 9, 1, 6, 3, 0, 2, 4, 5, 7, 10], [10, 2, 9, 0, 7, 1, 5, 6, 8, 3, 4, 10], [10], [], [], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10, 8, 9, 1, 6, 3, 0, 2, 4, 5, 7, 10], [10, 2, 9, 0, 7, 1, 5, 6, 8, 3, 4, 10], [10, 6, 5, 8, 9, 3, 2, 4, 1, 0, 7, 10], [10], [], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10, 8, 9, 1, 6, 3, 0, 2, 4, 5, 7, 10], [10, 2, 9, 0, 7, 1, 5, 6, 8, 3, 4, 10], [10, 6, 5, 8, 9, 3, 2, 4, 1, 0, 7, 10], [10, 3, 1, 6, 5, 2, 0, 9, 4, 8, 7, 10], [10], [], [], [], []]
[[10, 9, 8, 6, 7, 4, 0, 5, 1, 3, 2, 10], [10, 8, 9, 1, 6, 3, 0, 2, 4, 5, 7, 10], [10, 2, 9, 0, 7, 1, 5, 6, 8, 3, 4, 10], [10, 6, 5, 8, 9, 3, 2, 4, 1, 0, 7, 10], [10, 3, 1, 6, 5, 2, 0, 9, 4, 8, 7, 10], [

[[10], [], [], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10], [], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 0, 2, 4, 7, 10], [10], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 0, 2, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 7, 4, 0, 2, 10], [10], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 4, 0, 2, 7, 10], [10, 8, 9, 6, 5, 1, 3, 0, 2, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 7, 4, 0, 2, 10], [

[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 5, 6, 9, 8, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 3, 1, 5, 6, 9, 8, 4, 0, 2, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 5, 6, 9, 8, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 3, 1, 5, 6, 9, 8, 4, 0, 2, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10]]
Iteration #35:
Best fitness:  0.7780664375608553
Best tour:  [10, 7, 4, 0, 2, 5, 6, 9, 8, 1, 3, 10]
[[10], [], [], [], [], [], [], [], [], []]
[[10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10], [], [], [], [], [], [], [], []]
[[10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10],

[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10]

[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 7, 4, 0, 2, 3, 1, 5, 6, 9, 8, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10]]
Iteration #74:
Best fitness:  0.7780664375608553
Best tour:  [10, 7, 4, 0, 2, 5, 6, 9, 8, 1, 3, 10]
[[10], [], [], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10],

[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 10], [10], [], []]
[[10, 8, 9, 6, 5, 1, 3, 2, 0, 4, 7, 

In [201]:
# plt.savefig('sample.png')

In [202]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance=0 #total actual distance

for i in range(len(best_tour)-1):
    if best_tour[i]!=starting_point_index:
        coords_1 = (top_n_dests.iloc[best_tour[i]]["latitude"],top_n_dests.iloc[best_tour[i]]["longitude"])
        
        #names of source destination connected
        src=top_n_dests.iloc[best_tour[i]]["title"]
    else:
        coords_1 = (starting_point_lat,starting_point_long)
        src=starting_point_name

    if best_tour[i+1]!=starting_point_index:
        coords_2 = (top_n_dests.iloc[best_tour[i+1]]["latitude"],top_n_dests.iloc[best_tour[i+1]]["longitude"])
        dest=top_n_dests.iloc[best_tour[i+1]]["title"]
    else:
        coords_2 = (starting_point_lat,starting_point_long)
        dest=starting_point_name
        

    
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance=total_distance+distance

print("Total distance(km): ",total_distance)

Pātan(Home)->Namo Buddha (Stupa) 28.57494800498292
Namo Buddha (Stupa)->Kopan Monastery 28.601246483437034
Kopan Monastery->Boudhanath Stupa 2.343260122303754
Boudhanath Stupa->Pashupatinath Temple 1.7955626615260667
Pashupatinath Temple->Thamel 3.841106935293863
Thamel->Garden of Dreams 0.44689862662388247
Garden of Dreams->Asan 0.7649269678806467
Asan->Kathmandu Durbar Square 0.6745014662557417
Kathmandu Durbar Square->Swayambhunath Temple 2.0028041940669565
Swayambhunath Temple->Chandragiri Hills 5.473767406580083
Chandragiri Hills->Pātan(Home) 5.1214263709522
Total distance(km):  79.64044923990315
